In [2]:
import pandas as pd
import numpy as np
import glob

C:\Users\cyril\AppData\Local\Temp\ipykernel_3976\2742675143.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def phase_randomization(time_series):
    # Compute the Fourier transform of the time series
    fourier_transform = np.fft.fft(time_series)

    # Get the phases of the Fourier transform
    phases = np.angle(fourier_transform)

    # Shuffle the phases randomly
    np.random.shuffle(phases)

    # Apply the shuffled phases to the Fourier transform
    randomized_fourier_transform = np.abs(fourier_transform) * np.exp(1j * phases)

    # Reconstruct the randomized time series
    randomized_time_series = np.fft.ifft(randomized_fourier_transform)

    # Return the randomized time series
    return randomized_time_series.real

def phase_randomization_dataframe_fixed(df):
    df_dropped = df#.drop(df.columns[-1], axis=1)  # Drop the last column from the dataframe

    ##randomized_df = pd.DataFrame()  # Create an empty dataframe to store the randomized data
    L=[]
    for _, row in df_dropped.iterrows():
        randomized_row = phase_randomization(row.values)
        L.append(randomized_row)# Apply phase randomization to each row
        #randomized_df = randomized_df.concat(pd.Series(randomized_row), ignore_index=True)  # Add randomized row to the dataframe
    #randomized_df.columns = df_dropped.columns  # Assign the column names to the randomized dataframe
    #randomized_df=pd.concat(L)
    return pd.DataFrame(L)#randomized_df

In [8]:
path=glob.glob("Young_Adults_time_series/*.txt*")

In [ ]:
df=pd.read_csv(path[0],sep='\t',header=None)

In [ ]:
df=pd.read_csv('Cyril\\100206_rfMRI_REST1_Atlas_MSMAll_hp2000_clean_Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T.txt',sep='\t',header=None)

In [33]:
phase_randomization_dataframe(df)

,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
0,-0.108525,-0.044517,-0.084380,-0.047823,-0.015107,-0.058471,0.001609,0.081435,-0.082255,0.005339,...,0.085820,0.104227,0.045722,-0.032500,0.072316,0.055560,0.033321,0.039522,-0.024237,-0.093725
1,0.045132,0.089904,-0.078649,-0.043811,0.072687,-0.096808,0.056142,-0.109056,-0.079911,-0.017115,...,-0.025281,-0.056860,0.090380,0.015904,0.065626,-0.049170,0.085309,0.101431,0.042675,0.209763
2,-0.114612,0.080517,-0.000071,0.067246,0.095599,0.007831,0.061894,-0.047549,-0.000758,0.146837,...,-0.118224,0.052332,0.079373,-0.056687,0.124588,-0.079123,-0.007413,0.018465,0.028658,-0.035380
3,-0.110995,-0.018125,-0.121016,-0.043386,-0.081591,0.085252,-0.056239,0.007881,-0.110394,0.029552,...,0.040551,0.031759,0.111515,-0.003844,0.142761,0.130663,0.027556,0.065184,-0.006673,-0.065818
4,0.081476,0.123184,-0.003447,-0.154200,0.009673,-0.158991,-0.071204,-0.050077,-0.080510,0.132317,...,-0.087237,-0.090737,0.198301,0.119521,0.131238,0.029795,0.086181,0.083600,-0.096415,-0.104634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.071412,-0.086914,-0.019231,0.063397,0.051593,0.148845,0.288251,-0.081578,-0.072668,0.073256,...,-0.009346,0.070446,-0.012834,0.205826,0.119042,0.237178,0.089531,0.071480,0.180833,0.163822
112,0.166915,-0.195962,-0.163462,0.137532,-0.014491,0.019761,0.294385,0.090974,-0.116642,-0.010201,...,-0.057665,-0.089394,-0.203495,-0.123784,-0.149607,-0.214700,-0.204051,-0.045696,-0.207502,-0.234250
113,0.055465,-0.146443,-0.137001,-0.139444,-0.108761,-0.003800,-0.110391,-0.106778,-0.281915,-0.199740,...,-0.012000,-0.051688,-0.024969,-0.078564,-0.118040,-0.124188,-0.195152,-0.213289,-0.125790,-0.225560
114,0.096600,-0.289825,-0.099807,0.003970,0.051975,0.002501,0.034965,0.172558,0.222210,0.322499,...,-0.203612,0.213134,0.183010,0.019653,-0.051479,0.070855,0.053027,-0.003440,0.056852,-0.136693


In [36]:
for file in path:
    df=pd.read_csv(file,sep='\t',header=None)
    phase_randomization_dataframe_fixed(df)
    df.to_csv('Young_Adults_time_series_Phase_Random/PR_'+file.split("/")[-1])